# Import / Config

In [11]:
import pandas as pd
from sqlalchemy import create_engine
from neo4j import GraphDatabase

import cred_pg as c
import cred_neo4j as cc

## Postgres

In [12]:
engine = create_engine(
    f'postgresql+psycopg://{c.pg_userid}:{c.pg_password}@{c.pg_host}/{c.pg_db}', 
    connect_args = {
        'options': '-c search_path=${user},ugeobln,ugm,uinsta,umisc,umobility,usozmed,public', 
        'keepalives_idle': 120},
    pool_size=1, 
    max_overflow=0,
    execution_options={ 'isolation_level': 'AUTOCOMMIT' }
)

In [13]:
# engine.dispose()

In [14]:
with engine.connect() as con:
    haltestelle_pdf = pd.read_sql_query("select * from haltestelle", con)
    segment_pdf = pd.read_sql_query("select * from segment", con)
    linie_pdf = pd.read_sql_query("select * from linie", con)
    unterlinie_pdf = pd.read_sql_query("select * from unterlinie", con)
    abschnitt_pdf = pd.read_sql_query("select * from abschnitt", con)

In [15]:
abschnitt_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570 entries, 0 to 569
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ulid    570 non-null    int64 
 1   nr      570 non-null    int64 
 2   hid_a   570 non-null    int64 
 3   hid_b   570 non-null    int64 
 4   haelt   570 non-null    object
dtypes: int64(4), object(1)
memory usage: 22.4+ KB


## neo4j

In [16]:
cypher_create_stop = 'CREATE (h:Haltestelle {hid: $hid, bez: $bez, lat:$lat, lng:$lng})'
with GraphDatabase.driver(cc.neo4j_host, auth=(cc.neo4j_userid, cc.neo4j_password)) as driver:
    with driver.session() as session:
        for r in haltestelle_pdf.itertuples(index=False):
            session.run(cypher_create_stop, hid=r.hid, bez=r.bez, lat=r.lat, lng=r.lng)

In [17]:
cypher_create_linie = 'CREATE (l:Linie {lid: $lid, bez: $bez})'
with GraphDatabase.driver(cc.neo4j_host, auth=(cc.neo4j_userid, cc.neo4j_password)) as driver:
    with driver.session() as session:
        for r in linie_pdf.itertuples(index=False):
            session.run(cypher_create_linie, lid=r.lid, bez=r.bez)

In [18]:
cypher_create_segment = '''
MATCH (ha:Haltestelle), (hb:Haltestelle)
WHERE ha.hid=$hid_a AND hb.hid=$hid_b
CREATE (s:Segment {hid_a: ha.hid, hid_b: hb.hid, laengeInMeter: $laengeInMeter})
CREATE (s) -[:ProjSegA]-> (ha)
CREATE (s) -[:ProjSegB]-> (hb)
'''
with GraphDatabase.driver(cc.neo4j_host, auth=(cc.neo4j_userid, cc.neo4j_password)) as driver:
    with driver.session() as session:
        for r in segment_pdf.itertuples(index=False):
            session.run(cypher_create_segment, hid_a=r.hid_a, hid_b=r.hid_b, laengeInMeter=r.laenge_in_meter)

In [19]:
cypher_create_unterlinie = '''
MATCH (l:Linie)
WHERE l.lid=$lid
CREATE (ul:Unterlinie {ulid: $ulid})
CREATE (ul) -[:InL]-> (l)
'''
with GraphDatabase.driver(cc.neo4j_host, auth=(cc.neo4j_userid, cc.neo4j_password)) as driver:
    with driver.session() as session:
        for r in unterlinie_pdf.itertuples(index=False):
            session.run(cypher_create_unterlinie, ulid=r.ulid, lid=r.lid)

In [21]:
cypher_create_abschnitt = '''
MATCH (ha:Haltestelle), (hb:Haltestelle), (ul:Unterlinie)
WHERE ha.hid=$hid_a AND hb.hid=$hid_b AND ul.ulid=$ulid
CREATE (a:Abschnitt {nr: $nr, haelt: $haelt})
CREATE (a) -[:InUL]-> (ul)
CREATE (a) -[:ProjAbA]-> (ha)
CREATE (a) -[:ProjAbB]-> (hb)
'''
with GraphDatabase.driver(cc.neo4j_host, auth=(cc.neo4j_userid, cc.neo4j_password)) as driver:
    with driver.session() as session:
        for r in abschnitt_pdf.itertuples(index=False):
            session.run(cypher_create_abschnitt, ulid=r.ulid, nr=r.nr, hid_a=r.hid_a, hid_b=r.hid_b, haelt=r.haelt)